In [1]:
import pandas as pd
import re

In [ ]:
snippets = r"""

.       - Any Character Except New Line
\d      - Digit (0-9)
\D      - Not a Digit (0-9)
\w      - Word Character (a-z, A-Z, 0-9, _)
\W      - Not a Word Character
\s      - Whitespace (space, tab, newline)
\S      - Not Whitespace (space, tab, newline)

\b      - Word Boundary
\B      - Not a Word Boundary
^       - Beginning of a String
$       - End of a String

[]      - Matches Characters in brackets
[^ ]    - Matches Characters NOT in brackets
|       - Either Or
( )     - Group

Quantifiers:
*       - 0 or More
+       - 1 or More
?       - 0 or One
{3}     - Exact Number
{3,4}   - Range of Numbers (Minimum, Maximum)


#### Sample Regexs ####

[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+
"""

In [49]:
text = r"""
MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
\d 
\D 
MT5\MAM
ä90096
\123123
333333333
33333 

"""
print(text)


MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
\d 
\D 
MT5\MAM
ä90096
\123123
333333333
33333 




In [50]:
pattern = re.compile(r'333333333')
pattern

re.compile(r'333333333', re.UNICODE)

In [51]:
re.search(pattern, string = text) # return 1st match obj or None.
re.match(pattern, string = text)  # search at the beginning of string
re.split(pattern, string = text)

['\nMetaCharacters (Need to be escaped):\n. ^ $ * + ? { } [ ] \\ | ( )\n\\d \n\\D \nMT5\\MAM\nä90096\n\\123123\n',
 '\n33333 \n\n']

In [54]:
pattern = re.compile(r'33333')
re.findall(pattern, string = text)
match_obj = re.finditer(pattern, text)
for match in match_obj:
    print(match)

<re.Match object; span=(97, 102), match='33333'>
<re.Match object; span=(107, 112), match='33333'>


In [61]:
# ASCII 
text = r"""
ä90096
90096
"""
# Match ALL ASCII Printable Characters
# Match ALL ASCII Characters (including of Non-Printable Characters)
pattern = re.compile(r'[^ -~]')
pattern = re.compile(r'[^\x00-\x7F]')
matches = pattern.finditer(text)

for match in matches:
    print(match)


# print(re.sub(pattern = r'[^\x00-\x7F]', repl = '', string = text))

<re.Match object; span=(1, 2), match='ä'>

90096
90096



In [38]:
df = pd.DataFrame({
    'login' : [1, 2, 3, 4, 5, 6],
    'group' : ['VU_Hedge\MAM\MAM_VUM_100084_USD', 'VU_Hedge\MAM\MAM_VUM_100084_GBP', 'Moneta_Hedge\M_MNT_GBP', 'MNT_V_95054_GBP', 'SAM_VFX_MB_USD', 'M_MON_USD']
})
# Series.str.contains(pat, case=True, flags=0, na=None, regex=True)
df['group'].str.contains(pat = r'\\')
# Series.str.replace(pat, repl, n=- 1, case=None, flags=0, regex=None)
df['group'].str.replace(pat = r'\\', repl = '', regex = True)
# Series.str.extract(pat, flags=0, expand=True)
df['group'].str.extract(pat = r'([0-9]+)')   # 等價於r'(\d+)'
df['group'].str.extract(pat = r'(\d+)')  
# Exclude Pattern
df['group'].str.contains(pat = r'^((?!MAM).)*$')  
# step1.
df['group'].str.contains(pat = r'.*(?!MAM)')  




0    True
1    True
2    True
3    True
4    True
5    True
Name: group, dtype: bool

In [63]:
df = pd.read_excel('symbols.xlsx')
df.head()

,symbol,clean_symbol,symbol_type
0,ADAUSD,ADAUSD,Cryptocurrency
1,AUDCAD,AUDCAD,Forex
2,AUDCAD.i,AUDCAD,Forex
3,AUDCAD.ixp,AUDCAD,Forex
4,AUDCAD.uk+,AUDCAD,Forex


In [64]:
df['feature'] = df['symbol'].str.extract(r"([^A-Za-z0-9].*)", expand = False)
df['feature'].unique()

array([nan, '.i', '.ixp', '.uk+', '+', '.uk', '-OIL+', '#', '-OIL#',
       '.m#', '.m+', '.c', '&T', '.r', '.v', '-OIL', '-C', '-Cr', '.',
       '.Inst', '.pr', '-OIL.pr', '.pro', '.fr', '-OIL.fr', '.pr+',
       '-OIL.pr+', '.sp', '.m', '.a', '.s', '.e', '.y', '.r+', '.A',
       '.crp', '-OIL.e', '-OIL.i', '-OIL.s', '.m.e', '.p', '-OIL.p',
       '-Cs', '&T.m', '-ECN', '-STD', '-VIP', '.pr.m', '+.m', '.A.m',
       '.r.m', '-OIL-ECN', '-OIL-STD', '-OIL-VIP', '-OIL.m', '-OIL.pr.m',
       '-C.', '-C.m', '-Cr.m', '.crp.m', '-Cash', '.sc', '.b', '-OIL.',
       '-C.i', '-Cb'], dtype=object)

In [70]:
# replace(pat = r'\\', repl = '', regex = True)
df['feature2'] = df['feature'].str.replace(r'.*-OIL|.*.A|.*&T', repl = '', regex = True)
df['feature2'].unique()

array([nan, '.i', '.ixp', '.uk+', '+', '.uk', '#', '.m#', '.m+', '.c', '',
       '.r', '.v', '-C', '-Cr', '.', '.Inst', '.pr', '.pro', '.fr',
       '.pr+', '.sp', '.m', '.a', '.s', '.e', '.y', '.r+', '.crp', '.m.e',
       '.p', '-Cs', '-ECN', '-STD', '-VIP', '.pr.m', '+.m', '.r.m', '-C.',
       '-C.m', '-Cr.m', '.crp.m', '-Cash', '.sc', '.b', '-C.i', '-Cb'],
      dtype=object)

In [91]:
df['clean_symbol_regex'] = df.apply(lambda row : row['symbol'].replace(str(row['feature2']), ''), axis=1)
df_nonmatch = df[df['clean_symbol_regex'] != df['clean_symbol']]